# Libraries

In [1]:
# standard
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
from math import sqrt

# reading data
import os
import json
from collections import defaultdict

# machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.fft import rfft, irfft, fftn, ifftn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW

# visuals
import matplotlib.pyplot as plt
import seaborn as sns

# eFormer
from eFormer.embeddings import Encoding, ProbEncoding, PositionalEncoding
from eFormer.sparse_attention import ProbSparseAttentionModule, DetSparseAttentionModule
from eFormer.loss_function import crps
from eFormer.sparse_decoder import DetSparseDecoder, ProbSparseDecoder
from eFormer.Dataloader import TimeSeriesDataProcessor

%store -r Kelmarsh_df Penmanshiel_df

# Architektur

## Hyperparameters

In [2]:
# set global parameters

n_heads_global = 4
probabilistic_model = False
len_embedding_vector = 64

## Load Data

In [3]:
def shifted_data(data, forecast, look_back):
    data = data.set_index('# Date and time')
    data.index.names = [None]
    data = data.drop(['Long Term Wind (m/s)'], axis=1)
    shifts = range(forecast, look_back + forecast)
    variables = data.columns
        
    shifted_columns = []
    for column in variables:
        for i in shifts:
            shifted_df = data[[column]].shift(i)
            shifted_df.rename(columns={column: f"{column} (lag {i})"}, inplace=True)
            shifted_columns.append(shifted_df)
        
    data = data.drop(['Wind speed (m/s)'], axis=1)
    data_shifted = pd.concat([data] + shifted_columns, axis=1)
    data_shifted.dropna(inplace=True)
        
    return data_shifted

In [4]:
shifted_df = shifted_data(
    Kelmarsh_df['1'][-4096:],
    forecast=1,
    look_back=72
    )

%store shifted_df

Stored 'shifted_df' (DataFrame)


In [5]:
data = Kelmarsh_df['1'][-4096:]
data = data.set_index('# Date and time')
data.index.names = [None]
data = data.drop(['Long Term Wind (m/s)'], axis=1)

test_df = data

# Transformer Model

In [6]:
# Assuming `df` is your initial DataFrame
processor = TimeSeriesDataProcessor(
    dataframe=test_df,
    forecast=1,
    look_back=72,
    batch_size=64)
    
train_loader, test_loader, eval_loader = processor.create_dataloaders()

In [7]:
class eFormer(nn.Module):
    def __init__(self, in_features, len_embedding_vector, n_heads_global, probabilistic_model=False):
        super(eFormer, self).__init__()
        self.probabilistic_model = probabilistic_model
        self.n_heads_global = n_heads_global
        self.len_embedding_vector = len_embedding_vector

        # Initialize encoding model
        if probabilistic_model:
            self.encoding_model = ProbEncoding(in_features=in_features, out_features=len_embedding_vector)
        else:
            self.encoding_model = Encoding(in_features=in_features, out_features=len_embedding_vector)

        # Initialize attention module
        if probabilistic_model:
            self.attention_module = ProbSparseAttentionModule(d_model=len_embedding_vector, n_heads=n_heads_global, prob_sparse_factor=5)
        else:
            self.attention_module = DetSparseAttentionModule(d_model=len_embedding_vector, n_heads=n_heads_global, prob_sparse_factor=5)

        # Initialize decoder
        # Assuming the decoder initialization does not actually require the output shape directly but parameters that depend on the model configuration
        if probabilistic_model:
            self.decoder = ProbSparseDecoder(d_model=len_embedding_vector, n_heads=n_heads_global, forecast_horizon=1, encoder_output_dim=len_embedding_vector)
        else:
            self.decoder = DetSparseDecoder(d_model=len_embedding_vector, n_heads=n_heads_global, forecast_horizon=1, encoder_output_dim=len_embedding_vector)

    def forward(self, features_matrix):
        if torch.isnan(features_matrix).any():
            raise ValueError('NaN values detected in Input')

        embeddings = self.encoding_model(features_matrix)
        if torch.isnan(embeddings).any():
            raise ValueError('NaN values detected in Embeddings')

        encoder_output = self.attention_module(embeddings, embeddings, embeddings)
        if torch.isnan(encoder_output).any():
            raise ValueError('NaN values detected in Sparse Attention Output')

        forecasts, crps_weights = self.decoder(encoder_output)
        return forecasts, crps_weights

In [8]:
for batch in train_loader:
    features, labels = batch
    break

print(f"labels & features: {labels.shape, features.shape}")

%store features

labels & features: (torch.Size([64]), torch.Size([64, 144]))
Stored 'features' (Tensor)


In [11]:
model = eFormer(
    in_features=(features.shape[-1]),
    len_embedding_vector=64,
    n_heads_global=4,
    probabilistic_model=False)
optimizer = AdamW(
    params = model.parameters(),
    lr=6e-4,
    weight_decay=1e-1
    )
loss_fn = crps

num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        predictions, crps_weights = model(features)
        print(f"predictions type: {predictions.shape} \n crsp_weights type: {crps_weights.shape}")
        loss = loss_fn(forecast=predictions, observations=labels, weights=crps_weights)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch} / {num_epochs} with Loss: {loss}")

    # Evaluate your model's performance on the validation set
    model.eval()
    with torch.no_grad():
        for features, labels in test_loader:
            predictions = model(features)
            # Calculate and print validation metrics

predictions type: torch.Size([1, 64]) 
 crsp_weights type: torch.Size([1, 64, 64])


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

# Test Area

In [15]:
test_df.shape

(3878, 145)

In [ ]:
import torch
import numpy as np
import time

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0
            if self.verbose:
                print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
            self.val_loss_min = val_loss

early_stopping = EarlyStopping(patience=10, verbose=True)

for epoch in range(num_epochs):
    epoch_start_time = time.time()
    model.train()
    train_losses = []
    for features, labels in train_loader:
        optimizer.zero_grad()
        predictions = model(features)
        loss = loss_fn(predictions, labels)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
    
    train_loss_avg = np.mean(train_losses)
    print(f"Epoch {epoch} / {num_epochs} with Loss: {train_loss_avg}")

    # Validation phase
    model.eval()
    validation_losses = []
    with torch.no_grad():
        for features, labels in eval_loader:
            predictions = model(features)
            val_loss = loss_fn(predictions, labels)
            validation_losses.append(val_loss.item())

    val_loss_avg = np.mean(validation_losses)
    print(f"Validation Loss: {val_loss_avg}")

    epoch_end_time = time.time()
    epoch_duration = epoch_end_time - epoch_start_time
    print(f"Epoch Duration: {epoch_duration}s")

    early_stopping(val_loss_avg)
    if early_stopping.early_stop:
        print("Early stopping")
        break

In [47]:
def check_system_conditions():
    # Get CPU usage for each core
    cpu_percent = round(psutil.cpu_percent(), 4)

    # Get memory information
    memory_info = psutil.virtual_memory()
    memory_used_gb = round(memory_info.used / (1024 ** 3), 4)

    # Get GPU information
    try:
      gpu_info = GPUtil.getGPUs()[0]
      gpu_memory_used_gb = round(gpu_info.memoryUsed / 1024, 4)
    except IndexError:
      # If no GPU is found, set variables to None
      gpu_memory_used_gb = None

    # Collect data in a dictionary
    comp_usage = {
        'CPU Usage': cpu_percent,
        'Memory Usage (GB)': memory_used_gb,
        'GPU Usage (GB)': gpu_memory_used_gb
    }

    return comp_usage

In [ ]:
import time
import psutil
import GPUtil

# Assuming the check_system_conditions function is defined as previously mentioned

# Define a function to run monitoring in a separate thread
def monitor_system_usage(every_n_seconds=10, keep_running=lambda: True, results_list=[]):
    while keep_running():
        comp_usage = check_system_conditions()
        results_list.append(comp_usage)
        time.sleep(every_n_seconds)

# Initialize a list to store the results
system_usage_results = []

# Define a lambda function to control the monitoring loop
# It will return False to stop the thread once training is done
keep_monitoring = lambda: keep_monitoring_flag

# Initialize the flag to True before starting training
keep_monitoring_flag = True

# Start the monitoring thread
monitor_thread = threading.Thread(target=monitor_system_usage, args=(5, keep_monitoring, system_usage_results))
monitor_thread.start()

# Training loop here
# Insert your existing training loop code

# After training is done, set the flag to False to stop the monitoring thread
keep_monitoring_flag = False
monitor_thread.join()  # Wait for the monitoring thread to finish

# Convert the results list to a DataFrame
system_usage_df = pd.DataFrame(system_usage_results)

print(system_usage_df)
